In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset

# Load dataset
df = pd.read_csv('blog_samples.csv')  # Ensure the CSV contains the 'text' column
dataset = Dataset.from_pandas(df)

# Initialize LLaMA 2 tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('meta-llama/LLaMA-2-7b')
model = AutoModelForCausalLM.from_pretrained('meta-llama/LLaMA-2-7b')

# Tokenize data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,                  # Adjust based on need
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    logging_dir='./logs',
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Training
trainer.train()
trainer.save_model("llama_blog_generator")  # Save the fine-tuned model
tokenizer.save_pretrained("llama_blog_generator")
